# Setting up a mock DB to store Songs, users, and playlists
![](https://github.com/spotify-song/DS/blob/master/references/DB_tables_illu.png?raw=true)
### Tables:
    - User:
        - id
        - display_name
    - Tokens:
        - id
        - access_token
        - token_type
        - expries_in
        - refresh_token
        - scope
        - expires_at
        - user
    - Tracks
        - id
        - danceability
        - energy
        - loudness
        - mode
        - speechiness
        - accousticness
        - instrumentalness
        - valence
        - liveness
        - tempo
        - duration_ms
        - time_signature
    - UserPlaylist
        - id
        - u_id
        - uri
        - tracks_id

## DB import flow needs to be:
    - User > tokens
    - Tracks > top_tracks
    - users/tracks > user_playlists

### Get the pertinent data using the UserData() class

In [31]:
import sys
sys.path.insert(0, '../')
from spotify_users import UserData

## Plan:
    - check to see how we can use the access token and the refresh token to perform analysis.


In [1]:
from os import getenv
from dotenv import load_dotenv
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth

load_dotenv()

client_secret = getenv('SPOTIFY_CLIENT_SECRET')
client_id = getenv('SPOTIFY_CLIENT_ID')
scope = getenv('SCOPE')
uri = getenv('uri')

spot_oauth = spotipy.oauth2.SpotifyOAuth(
                                    client_id=client_id,
                                    client_secret=client_secret,
                                    scope=scope,
                                    redirect_uri=uri,
                                    )
spot_session = spotipy.Spotify(oauth_manager=spot_oauth)

In [10]:
token_info = spot_oauth.refresh_access_token()

In [9]:
token_info = spot_oauth.is_token_expired()